In [12]:
import zerorpc
import pandas as pd
from gensim.models import Word2Vec
from gensim.models import KeyedVectors 
list_csv = ['data/attraction_places.csv',
            'data/hotel_places.csv',
            'data/restaurant_places.csv']
models = ['model/attraction_tag_e5.model',
          'model/attraction_user.model',
          'model/hotel_tag_e5.model',
          'model/hotel_user.model',
          'model/restaurant_tag_e5.model',
          'model/restaurant_user.model']
class Recomm(object):
    def __init__(self):
        self.attraction_tag  = KeyedVectors.load(models[0])
        self.attraction_user = KeyedVectors.load(models[1])
        self.hotel_tag       = KeyedVectors.load(models[2])
        self.hotel_user      = KeyedVectors.load(models[3])
        self.restaurant_tag  = KeyedVectors.load(models[4])
        self.restaurant_user = KeyedVectors.load(models[5])
    def test(self):
        return 'test string'
    def rank(self, srcwv, entry, positive=[], negative=[]):
        subwv = srcwv[entry]
        newwv = Word2Vec(size=len(subwv[0]))
        # add entries to new vector
        for i in range(len(entry)):
            newwv.wv.add(entry[i], subwv[i])
        # add p, n to new vector
        for w in positive:
            newwv.wv.add(w, srcwv[w])
        for w in negative:
            newwv.wv.add(w, srcwv[w])
        ranked = newwv.most_similar(positive=positive,
                                    negative=negative,
                                    topn=len(entry))
        return ranked
    def drop(self, data):
        df = pd.read_csv(data, converters={'placeId':str})
        cols = df.columns.tolist()
        return df.drop(columns=cols[0:1]+cols[3:])
    def by_score(self, entry):
        df = pd.concat([self.drop(list_csv[i]) for i in range(3)])
        #cols = df.columns.tolist()
        #df = df.drop(columns=cols[0:1]+cols[3:])
        subdf = df[df['placeId'].isin(entry)]
        subdf = subdf.sort_values(by='score', ascending=False)
        ret = []
        for l in subdf.values:
            ret.append(tuple(l))
        if len(ret):
            raise Exception('Not in data')
        else:
            return ret
# tag, user 유사도 행렬 합한 wordvector로 rank하도록 수정할 것
    def recommend(self, group, entry, positive=[], negative=[]):
        # 선호 정보가 없을시 별점순으로 정렬
        if len(positive)==0 and len(negative)==0:
            return self.by_score(entry)
        if group == 'attractions':
            return self.rank(self.attraction_tag, entry, positive=positive, negative=negative)
        elif group == 'hotels':
            return self.rank(self.hotel_tag, entry, positive=positive, negative=negative)
        elif group == 'restaurants':
            return self.rank(self.restaurant_tag, entry, positive=positive, negative=negative)
        else:
            raise Exception('Unknown group')

In [6]:
entry = ['10020511', '10020529', '10020530']

In [18]:
entry = ['10686434', '10686449', '8749178']

In [ ]:
8822309

In [16]:
df = pd.read_csv(list_csv[0])
df

,class,placeId,score,name_kor,name_eng,location
0,attractions,10020510,5.000000,한성백제문화제,Hanseong Baekje Cultural Festival,"37.520847,127.11603"
1,attractions,10020511,4.750000,하이서울페스티벌,Hi Seoul Festival,"서울 (Multiple locations, including Seoul Plaza ..."
2,attractions,10020529,4.142857,서울불꽃축제,Seoul International Fireworks Festival,"37.528095,126.92417"
3,attractions,10020530,4.681818,서울빛초롱축제,Seoul Lantern Festival,"37.569157,127.00403"
4,attractions,10130671,5.000000,학아재미술관,Hagajae Museum,"37.577007,126.9904"
5,attractions,10183192,4.435897,석촌호수,Seokchon Lake,"37.508343,127.1"
6,attractions,10210051,3.200000,누죤,Nuzzon,서울 중구 을지로45길 62
7,attractions,10253198,4.400000,서울파이낸스센터,Seoul Finance Center,서울 중구 세종대로 136
8,attractions,10270564,4.187500,코엑스,COEX Center,서울 강남구 영동대로 513
9,attractions,10288072,4.300000,디뮤지엄,D Museum,"37.53687,127.01137"


In [19]:
r = Recomm()
r.recommend('attraction', entry, positive=['8822309'])

/home/jahn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
/home/jahn/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('8749178', 0.6300798058509827),
 ('10686449', 0.32585227489471436),
 ('10686434', -0.1834358125925064)]

In [8]:
r.by_score(entry)

[('10020511', 4.75),
 ('10020530', 4.681818181818182),
 ('10020529', 4.142857142857143)]

In [ ]:
try:
    server = zerorpc.Server(Recomm())
    server.bind("tcp://0.0.0.0:4242")
    server.run()
except Exception as e:
    print('exception caught')

/home/jahn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
/home/jahn/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
